# Library

In [1]:
# 標準ライブラリのインポート
import os
import json

# サードパーティライブラリのインポート
import requests
from dotenv import load_dotenv
import pandas_datareader.data as web 
import xml.etree.ElementTree as ET

# LLM関連のインポート
from openai import OpenAI

# APIキーの設定
load_dotenv() 
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# GPTモデルの設定
model_name='gpt-4o-mini-2024-07-18' #'gpt-4o-2024-08-06'

# 株価を取得する関数

In [2]:
# DataReaderでstooqから株価情報を取得する関数を定義（開始日～終了日）
def get_stock_price(code, date_from ,date_to):
    # 日本用のコードに変換
    code = code + '.JP'
    # stooqからデータ取得
    df = web.DataReader(code, data_source='stooq', start=date_from,end=date_to)
    # 日付と株価の終値を返す ※日付形式がおかしくなるのでorient='table'を利用
    return df['Close'].to_json(orient='table')

def get_stock_price_range(arguments):
    res = get_stock_price(
        code = arguments.get('code'),
        date_from = arguments.get('date_from'),
        date_to = arguments.get('date_to')
    )
    return res

# 天気を取得する関数

In [3]:
# 天気情報を取得する関数を定義
def get_info(latitude, longitude):
    url = "https://api.open-meteo.com/v1/forecast"
    parameters = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": "true"}
    response = requests.get(url, params=parameters)

    if response.status_code == 200:
        data = response.json()
        return json.dumps(data["current_weather"])
    else:
        return None

def get_weather_by_location(arguments):
    res = get_info(
        # 緯度と経度の情報 取得
        latitude = arguments.get('latitude'),
        longitude = arguments.get('longitude')
    )
    return res

# 書籍情報を取得する関数

In [4]:
# 書籍情報を取得する関数を定義
def get_books(keyword):
    url = "https://ci.nii.ac.jp/books/opensearch/search"
    parameters = {
        "q": keyword
    }
    response = requests.get(url, params=parameters)

    if response.status_code == 200:
        try:
            # Parse the XML response
            root = ET.fromstring(response.content)
            
            # Extract book titles and other relevant information
            books = []
            for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
                title = entry.find('{http://www.w3.org/2005/Atom}title').text
                link = entry.find('{http://www.w3.org/2005/Atom}link').attrib['href']
                author = entry.find('{http://purl.org/dc/elements/1.1/}publisher').text if entry.find('{http://purl.org/dc/elements/1.1/}publisher') is not None else 'Unknown'
                
                books.append({
                    "title": title,
                    "link": link,
                    "author": author
                })
            
            return json.dumps(books, ensure_ascii=False)  # Ensure correct encoding for Japanese characters
        except ET.ParseError as e:
            st.error("Failed to parse XML. Error:")
            st.write(str(e))
            return None
    else:
        st.error(f"Error: {response.status_code}")
        st.write(response.text)
        return None

def get_books_by_keyword(arguments):
    res = get_books(
        keyword=arguments.get('keyword')
    )
    return res

# Wikipedia情報を取得する関数を定義

In [5]:
# Wikipedia情報を取得する関数を定義
def get_wikipedia_info(keyword):
    url = f"https://ja.wikipedia.org/w/api.php"
    parameters = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "exintro": True,
        "explaintext": True,
        "titles": keyword
    }
    response = requests.get(url, params=parameters)

    if response.status_code == 200:
        data = response.json()
        pages = data["query"]["pages"]
        page_id = next(iter(pages))
        if page_id != "-1":
            return pages[page_id]["extract"]
        else:
            return "Wikipedia page not found for the keyword."
    else:
        return f"Error: {response.status_code}"

def get_wikipedia_by_keyword(arguments):
    res = get_wikipedia_info(
        keyword=arguments.get('keyword')
    )
    return res

# Function callingの設定

In [6]:
# 使用するツールのリスト
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price_range",  
            "description": "証券コードと2つの日付を渡すと指定の会社のその2つの日付の間の株価の終値を返します",  
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "証券コード",
                    },
                    "date_from": {
                        "type": "string",
                        "description": "株価を知りたい日付の開始日",
                    },
                    "date_to": {
                        "type": "string",
                        "description": "株価を知りたい日付の終了日",
                    },
                },
                "required": ["code", "date_from", "date_to"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather_by_location",  
            "description": "緯度と経度の情報から現在と将来の天気を取得",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "string",
                        "description": "緯度",
                    },
                    "longitude": {
                        "type": "string",
                        "description": "経度",
                    },
                },
                "required": ["latitude", "longitude"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_books_by_keyword",
            "description": "キーワードで書籍情報を検索し、関連する書籍情報を取得します",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "書籍情報を取得するための検索キーワード",
                    },
                },
                "required": ["keyword"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_wikipedia_by_keyword",
            "description": "キーワードで日本語Wikipediaから情報を取得します",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "Wikipediaで情報を取得するための検索キーワード",
                    },
                },
                "required": ["keyword"],
            },
        },
    },
]

# LLM

In [7]:
def llm_agent(user_input):

    client = OpenAI()

    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the supplied tools to assist the user if needed."},
        {"role": "user", "content": user_input}
    ]

    # ChatGPT API呼び出し
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice='auto',
    )

    # ツール呼び出しの結果があるか確認
    tool_calls = getattr(response.choices[0].message, 'tool_calls', None)  # tool_callsが存在しない場合はNone

    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name  # 属性としてアクセス
            arguments = json.loads(tool_call.function.arguments)

            # 関数の種類に基づいて処理を呼び出す
            if function_name == 'get_stock_price_range':
                function_response = get_stock_price_range(arguments)
            elif function_name == 'get_weather_by_location':
                function_response = get_weather_by_location(arguments)
            elif function_name == 'get_books_by_keyword':
                function_response = get_books_by_keyword(arguments)
            elif function_name == 'get_wikipedia_by_keyword':
                function_response = get_wikipedia_by_keyword(arguments)
            else:
                raise NotImplementedError(f"Unknown function: {function_name}")

            # ツール呼び出しごとに適切に応答を作成
            second_response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {'role': 'system', 'content': "You are a helpful assistant."},
                    {"role": "user", "content": user_input},
                    {"role": "assistant", "tool_call_id": tool_call.id, "content": ""},  # 空文字列を設定
                    {
                        'role': 'function',
                        'name': function_name,
                        'content': str(function_response),
                    },
                ]
            )
            # 最終的な応答を表示
            print(second_response.choices[0].message.content)
    else:
        # 関数が呼び出されていない場合、通常の応答を表示
        print(response.choices[0].message.content)


In [8]:
llm_agent('三井住友フィナンシャルグループの2024年8月の株価の動きと、9月の見通しについて分析してください。')

2024年8月の三井住友フィナンシャルグループの株価動向を分析すると、月初めから中旬にかけては大きな変動が見られました。具体的には、8月1日には10,880円という高値を記録した後、8月6日には8,000円まで急落しました。この急落の背景には、経済指標の発表や市場全体のセンチメントの変化が影響した可能性があります。

その後の動きでは、8月15日に9,699円、8月19日に10,040円といった上昇が見られたものの、再び8月23日には9,889円にまで下落し、一時的に9,000円台後半での推移が続きました。8月26日以降徐々に回復し、最終的には8月30日に9,550円で取引を終えています。

### 9月の展望

9月の株価見通しに関しては、以下のポイントが考えられます。

1. **経済指標の発表**: 9月には日本国内の経済指標（GDP、消費者信頼感指数など）が発表される予定です。それらの結果が株価に影響を与える可能性があります。特に、金融政策に関連する指標が注目されます。

2. **国内外の経済動向**: 世界経済、特にアメリカの利上げやインフレ動向は、投資家心理に影響を与えるでしょう。特に金融株は金利の影響を強く受けるため、利上げの可能性が高ければ、株価にプラスの影響を与えることも考えられます。

3. **テクニカル分析**: 現在の株価（9,550円）が、上昇トレンドに戻すことができるのかが注目されます。もし9,800円を超えてくるようであれば、更なる上昇が期待できるでしょう。

4. **市場全体のセンチメント**: 投資家の心理や他業種の動向も、三井住友フィナンシャルグループの株価に影響を与える要素となります。特に、金融システムの安定性やリスクへの感受性が高まる状況が続けば、慎重な動きが強まる可能性があります。

以上の要因を考慮しながら、9月の動きに注視したいと思います。引き続き市場の動向や経済指標に敏感に反応し、適切な投資判断を行うことが重要です。
三井住友フィナンシャルグループ（SMFG）の2024年8月および9月の株価動向を分析します。

### 2024年8月の株価動向
2024年8月の株価については具体的なデータが提供されていないため、一般的な市場の動きや経済状況を考慮する必要があります。この期間、国内外の経済指標や金利の動向、業界のニ

In [9]:
llm_agent('東京の天気を教えてください。')

現在の東京の天気は次の通りです：

- **気温**: 25.0°C
- **風速**: 2.1 m/s
- **風向き**: 121°（南東）
- **天候**: 快晴

もし今後の天気予報や詳細が必要であれば教えてください。


In [10]:
llm_agent('生成AIに関する最新の書籍を教えてください。')

以下は、最近出版された生成AIに関する書籍の一覧です。各書籍のリンクも合わせて掲載していますので、興味のあるものをぜひご覧ください。

1. [生成AIの論点 : 学問・ビジネスからカルチャーまで](https://ci.nii.ac.jp/ncid/BD07157185) - 青弓社
2. [生成AIパスポートテキスト&問題集 : GUGA公認](https://ci.nii.ac.jp/ncid/BD05525696) - 日本能率協会マネジメントセンター
3. [生成AI法務・ガバナンス : 未来を形作る規範](https://ci.nii.ac.jp/ncid/BD07709099) - 商事法務
4. [おとなもこどもも知りたい生成AIの教室](https://ci.nii.ac.jp/ncid/BD07706170) - カンゼン
5. [ChatGPT&画像生成AI](https://ci.nii.ac.jp/ncid/BD03282753) - インプレス
6. [Gemini完全マニュアル : Googleの対話型生成AI](https://ci.nii.ac.jp/ncid/BD07644528) - 秀和システム
7. [つくりながら学ぶ!生成AIアプリ&エージェント開発入門](https://ci.nii.ac.jp/ncid/BD07898730) - マイナビ出版
8. [AIで好きな絵をつくる!Stable Diffusion : 魔法の言葉、呪文(プロンプト)が満載!「画像生成AI」超入門](https://ci.nii.ac.jp/ncid/BD03703272) - 興陽館
9. [60分でわかる!生成AIビジネス活用最前線](https://ci.nii.ac.jp/ncid/BD06441660) - 技術評論社
10. [Open code interpreter自然言語生成AIコード開発](https://ci.nii.ac.jp/ncid/BD06990647) - [中山茂]
11. [生成AI : 社会を激変させるAIの創造力 = Generative AI](https://ci.nii.ac.jp/ncid/BD02336755) - SBクリエイティブ
12. [ChatGPT&生成AI : 倍速講義](htt

In [11]:
llm_agent('Wikipediaの情報をもとにLLMについて説明して下さい。')

ここでは、「LLM」が指す「大規模言語モデル」について説明します。

**大規模言語モデル (Large Language Model, LLM)** は、自然言語処理（NLP）の分野で使用される人工知能の一形態です。これらのモデルは、大量のテキストデータから学習し、自然言語を理解し、生成する能力を持っています。

### 特徴
1. **大規模なデータセット**:
   LLMは、インターネット上のテキストや書籍など、大量のデータで訓練されます。これにより、多様な文脈やテーマに関する知識を蓄積しています。

2. **トランスフォーマーアーキテクチャ**:
   多くのLLMはトランスフォーマーと呼ばれるニューラルネットワークのアーキテクチャを基にしています。この構造は、並列処理が可能であり、長文の依存関係を効率的に扱うことができます。

3. **文章生成能力**:
   訓練を通じて文脈を学習し、自然な形でテキストを生成する能力があります。質問応答、文章の要約、自動翻訳など、さまざまな応用が可能です。

4. **ファインチューニング**:
   特定のタスクや分野に特化するために、事前に訓練されたモデルを追加のデータで再教育（ファインチューニング）することができます。

### 利用例
- **チャットボット**:
  ユーザーとの対話を行い、情報を提供したり、コンサルティングサービスを提供したりします。
  
- **コンテンツ生成**:
  記事やストーリー、マーケティングコピーなど、さまざまなコンテンツを自動的に生成します。

- **翻訳**:
  異なる言語の間での翻訳タスクを高精度で実施します。

- **要約**:
  大量の情報を短い要約に凝縮する役割を果たします。

### 代表的なモデル
いくつかの著名な大規模言語モデルには、OpenAIのGPT（Generative Pre-trained Transformer）シリーズ、GoogleのBERT（Bidirectional Encoder Representations from Transformers）、MetaのLLaMA（Large Language Model Meta AI）などがあります。

大規模言語モデルは、さまざまな分野での応用が進んでおり、今後もさらなる進化が期待

In [12]:
llm_agent('LLMについて説明して下さい。')

LLMは、主に二つの意味で使われる略語です。一つは「大規模言語モデル（Large Language Model）」、もう一つは「法学修士（Master of Laws）」です。

### 大規模言語モデル (Large Language Model)
大規模言語モデルは、人工知能（AI）の一種で、膨大なテキストデータを使用して訓練されたモデルです。具体的には、文章の生成、要約、翻訳、質問応答など、さまざまな自然言語処理のタスクを行うことができます。これらのモデルは、深層学習（Deep Learning）技術を基にしており、多数のパラメータを持つため、非常に高い精度で人間のような自然な言語を理解し生成する能力があります。近年、GPT（Generative Pre-trained Transformer）シリーズやBERT（Bidirectional Encoder Representations from Transformers）などのモデルが広く知れ渡っています。

### 法学修士 (Master of Laws, LL.M.)
法学修士は、法学の大学院修士課程の一つです。この学位は、法律の専門家がさらなる専門知識を得るため、または国際的な法律の理解を深めるために取得されることが多いです。一般的には、法学の学士号（LL.B.）を持つ人が対象となります。LL.M.プログラムは、特定の法律分野（例えば、国際法、商法、知的財産法など）にフォーカスすることもあり、法律のキャリアアップや専門性の向上に寄与します。

このように、LLMは文脈に応じて異なる意味を持つ言葉ですので、使用する際には注意が必要です。
